In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
#hide
import json
from pathlib import Path
from typing import *
from getpass import getuser, getpass
from redminelib.resources import Issue
from fiscaliza.validation import valida_fiscaliza, auth_user
from fiscaliza.update import relatar_inspecao
from fiscaliza.info import detalhar_issue, extract_attachments, download_attachments
from fiscaliza.constants import TABLECOLS
from fastcore.foundation import L
from fastcore.script import Param
from rich.console import Console
console = Console()

Caso não desejar utilizar um ambiente virtual basta rodar:
`pip install fiscaliza`

In [7]:
# 
login = 'eric'
senha = getpass()
TESTE = True
issue = '103789'
parar_em = 'Aguardando Execução'
# API = 'bf5f7955ca65703bcc73afda4a5f215d0135bf2d'


In [8]:
detalhar_issue(issue, login, senha, teste=TESTE)

{'Anexos': {},
 'id': 103789,
 'subject': 'INSP_GR08_2023_0072',
 'status': <redminelib.resources.IssueStatus #1 "Rascunho">,
 'priority': <redminelib.resources.Enumeration #2 "Normal">,
 'start_date': '',
 'due_date': '',
 'description': '',
 'Data_de_inicio_efetivo': '',
 'Classe_da_Inspecao': '',
 'Tipo_de_Inspecao': '',
 'Ano': '2023',
 'Tema': ['{"valor":"Uso do Espectro","texto":"Uso do Espectro"}'],
 'Subtema': ['{"valor":"Radiomonitoração Terrestre","texto":"Radiomonitoração Terrestre"}'],
 'Coordenacao': 'FI2',
 'Numero_Sei_do_Processo': '{"numero"=>"53554.000002/2023-01", "link_acesso"=>"https://seihm.anatel.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1978467"}',
 'Fiscal_Responsavel': '',
 'Fiscais': [],
 'Entidade_da_Inspecao': [],
 'UF_Municipio': [],
 'Servicos_da_Inspecao': [],
 'Horas_de_Preparacao': '',
 'Horas_de_Deslocamento': '',
 'Horas_de_Execucao': '',
 'Horas_de_Conclusao': '',
 'Total_de_horas': '0.0',
 'SAV': '',
 'PCDP': '',
 'Proce

In [22]:
d = {}

d['Classe_da_Inspecao'] = 'Técnica' # str

d['Tipo_de_Inspecao'] = 'Uso do Espectro - Monitoração' #str

d['description'] = '''Realizar análise espectral de faixas das frequência relacionadas ao Plano de Monitoração do Espectro destinado a aplicações Críticas (PMEC), aferindo a sua ocupação, nas localidades que compõem a Etapa 1 da fiscalização.'''

d['Fiscal_Responsavel'] = 'Eric Magalhaes Delgado' #str

d['Fiscais'] = ['Alessandro Vasconcelos Fernandes', 'Eric Magalhaes Delgado'] #string ou lista de strings

# Windows
d['Html'] = r'arquivos/Report_2023.02.03_T14.08.31_85127.html' #str

# d['Html'] = r'D:\html_simples.html'

# #ou 

# #Unix d['Html'] = '/d/OneDrive - ANATEL/Monitoramento/53504.0005432021-55/Guarulhos.html' #str
            
d['Gerar_Relatorio'] = 1 # int 0 ou 1

d['Frequencia_Inicial']  = 10 #int ou float

d['Unidade_da_Frequencia_Inicial'] = 'kHz' #string

d['Frequencia_Final'] = 5450 #int ou float

d['Unidade_da_Frequencia_Final'] = 'MHz' #string

d['start_date'] = '2023-04-11' #YYYY-MM-DD #string nesse formato

d['due_date'] = '2023-05-31'  #YYYY-MM-DD #string nesse formato

d['UF_Municipio'] = "BA/Salvador" # string ou Lista de Strings: ["SP/São Paulo", "SP/Sorocaba"]

d['Servicos_da_Inspecao'] = ['019', '205',  '230', '231', '507', '805'] # String ou Lista de Strings

d['Qtd_Emissoes'] = 70 # int

d["Qtd_Licenciadas"] = 1

d["Qtd_Identificadas"] = 1

# d['Emissoes_nao_autorizadas'] = 24 # int

d['Horas_de_Preparacao'] = 4 # int

d['Horas_de_Deslocamento'] = 0 # int

d['Horas_de_Execucao'] = 4 # int

d['Horas_de_Conclusao'] = 16 # int

d['Latitude'] =  -23.635884 # float

d['Longitude'] = -46.654251 # float

d['Uso_de_PF'] = 'Não se aplica PF - uso apenas de formulários' # string

d['Acao_de_risco_a_vida_criada'] = 'Não' # string Sim | Não

d['Impossibilidade_acesso_online'] = '0' # string '0' | '1'

d['Reservar_Instrumentos'] = '0' #string '0' = Não | '1' = 'Sim'

d['Utilizou_algum_instrumento'] = '0'

# d['notes'] = """Vemos que o nível médio medido ficou da ordem de 35.2 dB inferior e o nível máximo da ordem de 25.2 dB inferior do valor esperado pela simulação de cobertura, indicando potenciais perdas adicionais não contabilizadas pelo modelo de propagação.

# Cabe salientar que concomitantemente à monitoração efetuada no item 4 foi efetuada a demodulação do sinal de áudio e monitoração do sinal ao vivo e não foi constatado qualquer sinal de interferência, batimento ou produto de intermodulação."""

# d['notes'] = ["""[{"Banda":"87.800 - 108.000 MHz", "Licenciados":31,"NaoLicenciados":4,"NaoLicenciaveis":0, 'Espúrio': 2, "NaoIdentificada": 1, "NaoManifestada": 0},
#                 {"Banda":"470.000 - 698.000 MHz","Sinais":19,"Licenciados":19,"NaoLicenciados":0,"NaoLicenciaveis":0, 'Espúrio': 2, "NaoIdentificada": 1, "NaoManifestada": 0}]
#             """]

# No caso de uma tabela, string formatada como csv

d['notes'] = {k:2 for k in TABLECOLS.values()}

d['notes'] = [f"{d['notes']}"]

# d['notes'] = ["""Faixa, Classe Especial, Classe A, Classe B, Classe C
#                 VHF-L,0,5,7,5
#                 VHF-H,0,12,1,0
#                 UHF,1,1,2,4
#                 FM,5,1,0,0
#                 RADCOM,0,0,0,0
#                 Outorgadas com indícios de irregularidades,1,2,3,4
#               """,
#               """[{"Banda":"87.800 - 108.000 MHz","Sinais":35,"Licenciados":31,"NaoLicenciados":4,"NaoLicenciaveis":0},
#                   {"Banda":"470.000 - 698.000 MHz","Sinais":19,"Licenciados":19,"NaoLicenciados":0,"NaoLicenciaveis":0}]
#               """]

d['uploads'] = {'path': r'arquivos/Report_2023.02.03_T14.08.31_85127.json', 
                'filename': "Info.json",
# #                 'description': "Lista de Emissões Desmascaradas e Não Licenciadas",
# #                 'content_type': '.csv'
                 }

In [23]:
relatar_inspecao(issue, login, senha, dados=d, teste=TESTE, parar_em=parar_em, substituir_relatorio=True)

Usuário Autenticado com Sucesso 👍

Inspeção 103789 vinculada à Ação 98268

Output()

✅ Estado Atual: Relatando

👩‍💻  A inspeção será atualizada até a situação Relatando

Output()

✨ Inspeção 103789 atualizada para Relatando

⚡  Relato efetuado

{'Anexos': {'Info.json': 'http://sistemasnethm/fiscaliza/attachments/download/4728/Info.json'},
 'id': 103789,
 'subject': 'INSP_GR08_2023_0072',
 'status': <redminelib.resources.IssueStatus #14 "Relatando">,
 'priority': <redminelib.resources.Enumeration #2 "Normal">,
 'start_date': datetime.date(2023, 4, 11),
 'due_date': datetime.date(2023, 5, 31),
 'description': 'Realizar análise espectral de faixas das frequência relacionadas ao Plano de Monitoração do Espectro destinado a aplicações Críticas (PMEC), aferindo a sua ocupação, nas localidades que compõem a Etapa 1 da fiscalização.',
 'Data_de_inicio_efetivo': datetime.date(2023, 2, 27),
 'Classe_da_Inspecao': 'Técnica',
 'Tipo_de_Inspecao': 'Uso do Espectro - Monitoração',
 'Ano': '2023',
 'Tema': ['{"valor":"Uso do Espectro","texto":"Uso do Espectro"}'],
 'Subtema': ['{"valor":"Radiomonitoração Terrestre","texto":"Radiomonitoração Terrestre"}'],
 'Coordenacao': 'FI2',
 'Numero_Sei_do_Processo': '{"numero"=>"53554.000002/2023-01", 

In [24]:
a = extract_attachments(issue, login, senha, teste=TESTE)
a

(#1) [<redminelib.resources.Attachment #4728 "Info.json">]

In [25]:
d['Html'] = r'arquivos/Report_2023.02.02_T23.14.07_84330.html' #str
d['uploads'] = {'path': r'arquivos/Report_2023.02.02_T23.14.07_84330.json', 
                'filename': "Info.json",
                'description': "Dados Consolidados da Inspeção"}

In [26]:
relatar_inspecao(issue, login, senha, dados=d, teste=TESTE, parar_em=parar_em, substituir_relatorio=True)

Usuário Autenticado com Sucesso 👍

Inspeção 103789 vinculada à Ação 98268

Output()

✅ Estado Atual: Relatando

👩‍💻  A inspeção será atualizada até a situação Relatando

Output()

✨ Inspeção 103789 atualizada para Relatando

⚡  Relato efetuado

{'Anexos': {'Info.json': 'http://sistemasnethm/fiscaliza/attachments/download/4729/Info.json'},
 'id': 103789,
 'subject': 'INSP_GR08_2023_0072',
 'status': <redminelib.resources.IssueStatus #14 "Relatando">,
 'priority': <redminelib.resources.Enumeration #2 "Normal">,
 'start_date': datetime.date(2023, 4, 11),
 'due_date': datetime.date(2023, 5, 31),
 'description': 'Realizar análise espectral de faixas das frequência relacionadas ao Plano de Monitoração do Espectro destinado a aplicações Críticas (PMEC), aferindo a sua ocupação, nas localidades que compõem a Etapa 1 da fiscalização.',
 'Data_de_inicio_efetivo': datetime.date(2023, 2, 27),
 'Classe_da_Inspecao': 'Técnica',
 'Tipo_de_Inspecao': 'Uso do Espectro - Monitoração',
 'Ano': '2023',
 'Tema': ['{"valor":"Uso do Espectro","texto":"Uso do Espectro"}'],
 'Subtema': ['{"valor":"Radiomonitoração Terrestre","texto":"Radiomonitoração Terrestre"}'],
 'Coordenacao': 'FI2',
 'Numero_Sei_do_Processo': '{"numero"=>"53554.000002/2023-01", 

In [27]:
a = extract_attachments(issue, login, senha, teste=TESTE)
a

(#1) [<redminelib.resources.Attachment #4729 "Info.json">]